# Find Bugs

Use this notebook to find wrong predicitons. We can then later visualize the pairs of specs.

In [ ]:
from draco.learn import data_util
from draco.learn import linear
from sklearn import linear_model
import random
import json
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
pos_neg_data = data_util.get_pos_neg_data()
X, y = linear.prepare_data(pos_neg_data)

clf = linear_model.LogisticRegression(solver='sag')

clf.fit(X, y)

print("Score: ", clf.score(X, y))

In [ ]:
# get the pairs that are predicted wrong
negative_pairs = X[y == 0]

predicted = clf.predict(negative_pairs)
bug_idx = predicted != 0  # idx in pairs

np.arange(len(bug_idx))[bug_idx]

In [ ]:
# sanity check, these should be the same as the bugs for positive pairs
positive_pairs = X[y == 1]

predicted = clf.predict(positive_pairs)
bug_idx = predicted != 1

np.arange(len(bug_idx))[bug_idx]

In [ ]:
# get the indexes in the original data
bugs = train.index[bug_idx]
bugs

In [ ]:
raw_data = data_util.get_raw_data()

In [ ]:
# TODO: generate better data

vals = [{
    'q1': random.normalvariate(2, 2),
    'q2': random.normalvariate(2, 2),
    'n': random.randint(0,7)
} for _ in range(10)]

In [ ]:
bug_specs = []

for i in bugs:
    schema, negative, positive = raw_data[i]
    negative['data'] = {
        'values': vals
    }
    positive['data'] = {
        'values': vals
    }
    bug_specs.append({
        'negative': negative,
        'positive': positive
    })

In [ ]:
with open('../data/bugs/bugs.json', 'w') as f:
    json.dump(bug_specs, f, indent=2)